<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/01_preprocessing/DF_real_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
from google.colab import drive

# 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def gaussian_blur(img, level):
    # [0.3, 0.6, 0.9, 1.2, 1.5] (너무 약함)
    # [1.0, 1.5, 2.2, 3.0, 4.0] (너무 강함)
    # 위 두 수치의 중간값으로 보정
    sigmas = [0.7, 1.1, 1.6, 2.1, 2.8]
    sigma = sigmas[level-1]

    return cv2.GaussianBlur(img, (0, 0), sigma)

def gaussian_noise(img, level):
    # 기존 [10, 20, 35, 50, 70]에서 전체적으로 강도를 살짝 낮춤
    # [Level 1, 2, 3, 4, 5]
    sigmas = [7, 15, 25, 40, 55]
    sigma = sigmas[level-1]

    h, w, c = img.shape

    # 입자 크기(Grain Size) 보정 로직은 유지 (거친 입자감 재현)
    noise_h, noise_w = h // 2, w // 2
    noise = np.random.normal(0, sigma, (noise_h, noise_w, c))

    # Nearest Neighbor 보간법으로 확대
    noise_resized = cv2.resize(noise, (w, h), interpolation=cv2.INTER_NEAREST)

    # 원본 이미지에 합성
    out = img.astype(np.float32) + noise_resized
    return np.clip(out, 0, 255).astype(np.uint8)

def jpeg_compression(img, level):
    # 기존 [50, 35, 25, 15, 8]에서 하향 조정
    # Level 3에서 Quality 15 정도면 확실한 격자 무늬(Block artifact)가 발생합니다.
    qualities = [35, 25, 15, 10, 5]
    quality = qualities[level-1]

    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, encimg = cv2.imencode('.jpg', img, encode_param)
    return cv2.imdecode(encimg, 1)

def block_wise(img, level):
    """지적하신 '회색 픽셀/국소 블록' 양상을 재현하는 함수"""
    out = img.copy()
    h, w = img.shape[:2]
    # 레벨에 따라 블록의 개수와 크기를 조절 (256px 기준 최적화)
    block_counts = [3, 8, 15, 25, 40]
    block_sizes = [4, 8, 12, 16, 20]

    count = block_counts[level-1]
    size = block_sizes[level-1]

    for _ in range(count):
        y, x = np.random.randint(0, h - size), np.random.randint(0, w - size)
        # f000_1.jpg처럼 회색 계열의 매크로블록 에러 시뮬레이션
        if np.random.rand() > 0.5:
            out[y:y+size, x:x+size] = 128 # 회색 블록
        else:
            # 주변부를 뭉개는 블록
            roi = out[y:y+size, x:x+size]
            out[y:y+size, x:x+size] = cv2.blur(roi, (size, size))
    return out

def color_contrast(img, level):
    # 너무 어두웠던 수치와 너무 밝았던 수치의 중간값으로 조정했습니다.
    # [Level 1, 2, 3, 4, 5]
    factors = [0.85, 0.75, 0.65, 0.55, 0.45]
    factor = factors[level-1]

    # 원본 방식을 유지하여 밝기와 대비를 동시에 조절
    out = img.astype(np.float32) * factor

    return np.clip(out, 0, 255).astype(np.uint8)

def color_saturation(img, level):
    # Level 3에서 거의 흑백이 되도록 수치를 대폭 하향 조정
    # [Level 1, 2, 3, 4, 5]
    factors = [0.60, 0.40, 0.15, 0.05, 0.00]
    factor = factors[level-1]

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    # 채도(S) 채널에 factor 적용 (0.15 수준이면 거의 흑백으로 보임)
    hsv[:, :, 1] *= factor

    return cv2.cvtColor(np.clip(hsv, 0, 255).astype(np.uint8), cv2.COLOR_HSV2BGR)

def video_compression(img, level):
    # VC는 JPEG보다 더 뭉개지도록 아주 낮은 수치를 적용합니다.
    # Level 5의 2는 공식 데이터셋의 가장 강력한 압축 수준입니다.
    qualities = [30, 20, 10, 5, 2]
    quality = qualities[level-1]

    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, encimg = cv2.imencode('.jpg', img, encode_param)
    return cv2.imdecode(encimg, 1)

def apply_perturbation(img, dist_type, level):
    if dist_type == 'GB': return gaussian_blur(img, level)
    elif dist_type == 'GNC': return gaussian_noise(img, level)
    elif dist_type == 'JPEG': return jpeg_compression(img, level)
    elif dist_type == 'BW': return block_wise(img, level)
    elif dist_type == 'CC': return color_contrast(img, level)
    elif dist_type == 'CS': return color_saturation(img, level)
    elif dist_type == 'VC': return video_compression(img, level)
    return img

In [3]:
# 압축풀기
!unzip -q /content/drive/MyDrive/HECTO/Dataset/images_data.zip -d /content/data_local

In [4]:
# --- 경로 설정 ---
# 1. 원본 FF++ Real 사진 경로
FF_REAL_BASE = '/content/data_local/content/drive/MyDrive/HECTO/Dataset/Images/FF_frames/Real'
# 2. 메타 파일(.txt)이 들어있는 경로
META_ROOT = '/content/drive/MyDrive/HECTO/Dataset/DF_lists'
# 3. 결과물을 저장할 DF Real 경로
SAVE_BASE = '/content/drive/MyDrive/HECTO/Dataset/DF_Frames_Final/Real'

# --- 사용자 요청 설정 (ID 범위 및 레벨) ---
configs = [
    ('manipulated_videos_end_to_end_level_3_meta.txt', range(0, 500), 'level_3'),
    ('manipulated_videos_end_to_end_level_4_meta.txt', range(500, 1000), 'level_4'),
    ('manipulated_videos_end_to_end_level_5_meta.txt', range(0, 500), 'level_5'),
    ('manipulated_videos_end_to_end_mix_2_distortions_meta.txt', range(500, 1000), 'mix_2')
]

def load_dist_map(meta_path, id_range):
    dist_map = {}
    if not os.path.exists(meta_path):
        return dist_map
    with open(meta_path, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')
            if len(parts) < 2: continue

            # 아이디 추출
            v_id_str = os.path.basename(parts[0]).split('_')[0]

            try:
                if int(v_id_str) in id_range:
                    # 모든 왜곡 정보(CS:3, VC:5 등)를 리스트로 수집
                    dist_list = []
                    for d_str in parts[1:]:
                        if ':' in d_str:
                            d_type, d_level = d_str.split(':')
                            dist_list.append((d_type, int(d_level)))
                    dist_map[v_id_str] = dist_list
            except: continue
    return dist_map

In [5]:
# --- 전처리 실행 루프 (configs 설정 반영 버전) ---

# configs 설정에 따라 순차적으로 실행
for meta_file, id_range, dist_folder_name in configs:
    meta_path = os.path.join(META_ROOT, meta_file)
    print(f"\n📂 작업 시작: {dist_folder_name} (ID 범위: {id_range.start}~{id_range.stop-1})")

    # configs에 정의된 id_range를 인자로 전달
    current_dist_map = load_dist_map(meta_path, id_range)

    if not current_dist_map:
        print(f"⚠️ {meta_file} 로드 실패 또는 해당 범위에 데이터 없음. 건너뜁니다.")
        continue

    # 아이디별 처리 루프
    for v_id, distortions in tqdm(current_dist_map.items(), desc=f"Processing {dist_folder_name}"):
        src_folder = os.path.join(FF_REAL_BASE, v_id)
        if not os.path.exists(src_folder): continue

        dst_folder = os.path.join(SAVE_BASE, dist_folder_name, f"{v_id}_{dist_folder_name}")

        # 이미 15장 이상 처리되었다면 건너뛰기
        if os.path.exists(dst_folder) and len(os.listdir(dst_folder)) >= 15: continue
        os.makedirs(dst_folder, exist_ok=True)

        frames = sorted([f for f in os.listdir(src_folder) if f.lower().endswith(('.jpg', '.png'))])[:15]

        for img_name in frames:
            img = cv2.imread(os.path.join(src_folder, img_name))
            if img is None: continue

            distorted_img = img.copy()
            for d_type, d_level in distortions:
                distorted_img = apply_perturbation(distorted_img, d_type, d_level)

            cv2.imwrite(os.path.join(dst_folder, img_name), distorted_img)

print("\n✨ 설정된 범위(레벨별 500개)에 따른 전처리가 완료되었습니다.")


📂 작업 시작: level_3 (ID 범위: 0~499)


Processing level_3: 100%|██████████| 500/500 [02:14<00:00,  3.71it/s]



📂 작업 시작: level_4 (ID 범위: 500~999)


Processing level_4: 100%|██████████| 500/500 [02:13<00:00,  3.75it/s]



📂 작업 시작: level_5 (ID 범위: 0~499)


Processing level_5: 100%|██████████| 500/500 [02:11<00:00,  3.80it/s]



📂 작업 시작: mix_2 (ID 범위: 500~999)


Processing mix_2: 100%|██████████| 500/500 [02:30<00:00,  3.33it/s]


✨ 설정된 범위(레벨별 500개)에 따른 전처리가 완료되었습니다.


In [6]:
# # 왜곡 강도 확인용
# # --- 설정 ---
# TEST_ID = '012'  # 비교해보고 싶은 아이디
# TEST_LEVEL = 3   # 확인하고 싶은 왜곡 레벨
# SAMPLE_SRC = os.path.join(FF_REAL_BASE, TEST_ID)
# SAMPLE_SAVE_DIR = '/content/drive/MyDrive/HECTO/Dataset/DF_Test_Samples'
# os.makedirs(SAMPLE_SAVE_DIR, exist_ok=True)

# # 왜곡 타입 리스트
# dist_types = ['VC']

# # 원본 이미지 로드
# frame_list = sorted([f for f in os.listdir(SAMPLE_SRC) if f.lower().endswith(('.jpg', '.png'))])

# if not frame_list:
#     print(f"⚠️ {TEST_ID} 폴더에 이미지가 없습니다.")
# else:
#     sample_img_name = frame_list[0]
#     img_path = os.path.join(SAMPLE_SRC, sample_img_name)
#     original_img = cv2.imread(img_path)

#     print(f"🚀 ID: {TEST_ID} | 프레임: {sample_img_name} 샘플 생성을 시작합니다.")

#     # 1. 원본 저장 (ID 명시)
#     original_save_name = f"{TEST_ID}_ORIGINAL_{sample_img_name}"
#     cv2.imwrite(os.path.join(SAMPLE_SAVE_DIR, original_save_name), original_img)
#     print(f"✅ 원본 저장 완료: {original_save_name}")

#     # 2. 왜곡별 적용 및 저장 (ID와 조작방식 명시)
#     for d_type in dist_types:
#         distorted = apply_perturbation(original_img.copy(), d_type, TEST_LEVEL)

#         # 파일명 규칙: ID_조작방식_레벨_파일명
#         save_name = f"{TEST_ID}_{d_type}_LV{TEST_LEVEL}_{sample_img_name}"
#         save_path = os.path.join(SAMPLE_SAVE_DIR, save_name)

#         cv2.imwrite(save_path, distorted)
#         print(f"✅ 생성 완료: {save_name}")

#     print(f"\n✨ 모든 샘플이 {SAMPLE_SAVE_DIR} 에 저장되었습니다.")